In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1633559,2021-09-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1633560,2021-09-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1633561,2021-09-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1633562,2021-09-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
42747,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42749,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42751,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42753,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42755,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
996675,2021-09-07,Arkansas,Arkansas,5001,3191,54.00,5000,Arkansas,AR,Arkansas,State,3017804
996677,2021-09-08,Arkansas,Arkansas,5001,3195,54.00,5000,Arkansas,AR,Arkansas,State,3017804
996679,2021-09-09,Arkansas,Arkansas,5001,3204,54.00,5000,Arkansas,AR,Arkansas,State,3017804
996681,2021-09-10,Arkansas,Arkansas,5001,3224,54.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1633559,2021-09-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1633560,2021-09-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1633561,2021-09-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1633562,2021-09-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-09-11') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
599,2021-09-11,Snohomish,Washington,53814,665.00,53061,WA,County,822083,6546.05,80.89
1196,2021-09-11,Cook,Illinois,602648,11405.00,17031,IL,County,5150233,11701.37,221.45
1792,2021-09-11,Orange,California,310195,5302.00,6059,CA,County,3175692,9767.79,166.96
2387,2021-09-11,Maricopa,Arizona,663073,11013.00,4013,AZ,County,4485414,14782.87,245.53
2982,2021-09-11,Los Angeles,California,1428526,25630.00,6037,CA,County,10039107,14229.61,255.30
...,...,...,...,...,...,...,...,...,...,...,...
1632351,2021-09-11,Wheeler,Oregon,62,1.00,41069,OR,County,1332,4654.65,75.08
1632685,2021-09-11,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1632988,2021-09-11,Esmeralda,Nevada,45,0.00,32009,NV,County,873,5154.64,0.00
1633287,2021-09-11,Loving,Texas,7,0.00,48301,TX,County,169,4142.01,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
599,2021-09-11,Snohomish,Washington,53814,665.00,53061,WA,County,822083,6546.05,80.89,80.89,6546.05
1196,2021-09-11,Cook,Illinois,602648,11405.00,17031,IL,County,5150233,11701.37,221.45,221.45,11701.37
1792,2021-09-11,Orange,California,310195,5302.00,6059,CA,County,3175692,9767.79,166.96,166.96,9767.79
2387,2021-09-11,Maricopa,Arizona,663073,11013.00,4013,AZ,County,4485414,14782.87,245.53,245.53,14782.87
2982,2021-09-11,Los Angeles,California,1428526,25630.00,6037,CA,County,10039107,14229.61,255.30,255.30,14229.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1632351,2021-09-11,Wheeler,Oregon,62,1.00,41069,OR,County,1332,4654.65,75.08,75.08,4654.65
1632685,2021-09-11,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1632988,2021-09-11,Esmeralda,Nevada,45,0.00,32009,NV,County,873,5154.64,0.00,0.00,5154.64
1633287,2021-09-11,Loving,Texas,7,0.00,48301,TX,County,169,4142.01,0.00,0.00,4142.01


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1042432,2021-09-11,Hale,Alabama,2798,81.00,1065,AL,County,14651,19097.67,552.86,552.86,19097.67,1
1135277,2021-09-11,Clarke,Alabama,4410,68.00,1025,AL,County,23622,18669.04,287.87,287.87,18669.04,2
721859,2021-09-11,Etowah,Alabama,18145,410.00,1055,AL,County,102268,17742.60,400.91,400.91,17742.60,3
605256,2021-09-11,Franklin,Alabama,5528,92.00,1059,AL,County,31362,17626.43,293.35,293.35,17626.43,4
722931,2021-09-11,Marshall,Alabama,16736,250.00,1095,AL,County,96774,17293.90,258.33,258.33,17293.90,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877176,2021-09-11,Johnson,Wyoming,975,13.00,56019,WY,County,8445,11545.29,153.94,153.94,11545.29,19
1379535,2021-09-11,Niobrara,Wyoming,248,3.00,56027,WY,County,2356,10526.32,127.33,127.33,10526.32,20
1238239,2021-09-11,Lincoln,Wyoming,2004,14.00,56023,WY,County,19830,10105.90,70.60,70.60,10105.90,21
995087,2021-09-11,Sublette,Wyoming,984,11.00,56035,WY,County,9831,10009.15,111.89,111.89,10009.15,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
814581,2021-09-11,Lowndes,Alabama,1657,55.00,1085,AL,County,9726,17036.81,565.49,565.49,17036.81,6,1
1042432,2021-09-11,Hale,Alabama,2798,81.00,1065,AL,County,14651,19097.67,552.86,552.86,19097.67,1,2
878778,2021-09-11,Greene,Alabama,1171,41.00,1063,AL,County,8111,14437.18,505.49,505.49,14437.18,47,3
345236,2021-09-11,Walker,Alabama,9800,302.00,1127,AL,County,63521,15427.97,475.43,475.43,15427.97,33,4
721323,2021-09-11,Dallas,Alabama,4518,173.00,1047,AL,County,37196,12146.47,465.10,465.10,12146.47,61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552677,2021-09-11,Weston,Wyoming,829,6.00,56045,WY,County,6927,11967.66,86.62,86.62,11967.66,17,19
1207146,2021-09-11,Uinta,Wyoming,3204,16.00,56041,WY,County,20226,15841.00,79.11,79.11,15841.00,4,20
1238239,2021-09-11,Lincoln,Wyoming,2004,14.00,56023,WY,County,19830,10105.90,70.60,70.60,10105.90,21,21
811371,2021-09-11,Albany,Wyoming,5391,23.00,56001,WY,County,38880,13865.74,59.16,59.16,13865.74,9,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.89,6546.05,24,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.89,6546.05,24,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.89,6546.05,24,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.89,6546.05,24,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.89,6546.05,24,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629765,2021-09-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1629766,2021-09-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1629767,2021-09-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1629768,2021-09-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1038108,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19097.67,2,1,1.00
1038109,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19097.67,2,1,0.00
1038110,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19097.67,2,1,0.00
1038111,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19097.67,2,1,0.00
1038112,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,552.86,19097.67,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259163,2021-09-07,Crook,Wyoming,601,13.00,56011,WY,County,7584,7924.58,171.41,171.41,8254.22,9,23,27.00
1259164,2021-09-08,Crook,Wyoming,611,13.00,56011,WY,County,7584,8056.43,171.41,171.41,8254.22,9,23,10.00
1259165,2021-09-09,Crook,Wyoming,619,13.00,56011,WY,County,7584,8161.92,171.41,171.41,8254.22,9,23,8.00
1259166,2021-09-10,Crook,Wyoming,626,13.00,56011,WY,County,7584,8254.22,171.41,171.41,8254.22,9,23,7.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
811319,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17036.81,1,6,1.00,0.00
811320,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17036.81,1,6,0.00,0.00
811321,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17036.81,1,6,0.00,0.00
811322,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17036.81,1,6,0.00,0.00
811323,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,17036.81,1,6,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341414,2021-09-07,Teton,Wyoming,4568,11.00,56039,WY,County,23464,19468.12,46.88,46.88,19651.38,23,1,66.00,0.00
341415,2021-09-08,Teton,Wyoming,4572,11.00,56039,WY,County,23464,19485.17,46.88,46.88,19651.38,23,1,4.00,0.00
341416,2021-09-09,Teton,Wyoming,4587,11.00,56039,WY,County,23464,19549.10,46.88,46.88,19651.38,23,1,15.00,0.00
341417,2021-09-10,Teton,Wyoming,4611,11.00,56039,WY,County,23464,19651.38,46.88,46.88,19651.38,23,1,24.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-09-11') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
412412,2021-09-11,Imperial,California,33763,750.00,6025,CA,County,181215,18631.46,413.87,413.87,18631.46,1,3,55.00,0.00
2982,2021-09-11,Los Angeles,California,1428526,25630.00,6037,CA,County,10039107,14229.61,255.30,255.30,14229.61,2,8,0.00,19.00
204826,2021-09-11,San Bernardino,California,342248,5422.00,6071,CA,County,2180085,15698.84,248.71,248.71,15698.84,3,4,854.00,5.00
91337,2021-09-11,Stanislaus,California,78021,1194.00,6099,CA,County,550660,14168.63,216.83,216.83,14168.63,4,9,0.00,0.00
723955,2021-09-11,Inyo,California,1599,39.00,6027,CA,County,18039,8864.13,216.20,216.20,8864.13,5,35,4.00,0.00
72605,2021-09-11,San Joaquin,California,94941,1573.00,6077,CA,County,762148,12457.03,206.39,206.39,12457.03,6,16,334.00,3.00
42192,2021-09-11,Riverside,California,352267,4826.00,6065,CA,County,2470546,14258.67,195.34,195.34,14258.67,7,7,1214.00,0.00
655773,2021-09-11,Merced,California,38304,527.00,6047,CA,County,277680,13794.30,189.79,189.79,13794.30,8,10,0.00,0.00
113334,2021-09-11,Tulare,California,70089,883.00,6107,CA,County,466195,15034.27,189.41,189.41,15034.27,9,5,366.00,3.00
41084,2021-09-11,Fresno,California,132281,1846.00,6019,CA,County,999101,13240.00,184.77,184.77,13240.00,10,13,619.00,8.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1541761,2021-09-11,Lassen,California,6577,31.00,6035,CA,County,30573,21512.45,101.40,101.40,21512.45,32,1,0.00,0.00
878720,2021-09-11,Kings,California,29078,274.00,6031,CA,County,152940,19012.68,179.16,179.16,19012.68,11,2,191.00,0.00
412412,2021-09-11,Imperial,California,33763,750.00,6025,CA,County,181215,18631.46,413.87,413.87,18631.46,1,3,55.00,0.00
204826,2021-09-11,San Bernardino,California,342248,5422.00,6071,CA,County,2180085,15698.84,248.71,248.71,15698.84,3,4,854.00,5.00
113334,2021-09-11,Tulare,California,70089,883.00,6107,CA,County,466195,15034.27,189.41,189.41,15034.27,9,5,366.00,3.00
257733,2021-09-11,Kern,California,130375,1496.00,6029,CA,County,900202,14482.86,166.18,166.18,14482.86,14,6,0.00,0.00
42192,2021-09-11,Riverside,California,352267,4826.00,6065,CA,County,2470546,14258.67,195.34,195.34,14258.67,7,7,1214.00,0.00
2982,2021-09-11,Los Angeles,California,1428526,25630.00,6037,CA,County,10039107,14229.61,255.30,255.30,14229.61,2,8,0.00,19.00
91337,2021-09-11,Stanislaus,California,78021,1194.00,6099,CA,County,550660,14168.63,216.83,216.83,14168.63,4,9,0.00,0.00
655773,2021-09-11,Merced,California,38304,527.00,6047,CA,County,277680,13794.30,189.79,189.79,13794.30,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
412412,2021-09-11,Imperial,California,33763,750.00,6025,CA,County,181215,18631.46,413.87,413.87,18631.46,1,3,55.00,0.00
2982,2021-09-11,Los Angeles,California,1428526,25630.00,6037,CA,County,10039107,14229.61,255.30,255.30,14229.61,2,8,0.00,19.00
204826,2021-09-11,San Bernardino,California,342248,5422.00,6071,CA,County,2180085,15698.84,248.71,248.71,15698.84,3,4,854.00,5.00
91337,2021-09-11,Stanislaus,California,78021,1194.00,6099,CA,County,550660,14168.63,216.83,216.83,14168.63,4,9,0.00,0.00
723955,2021-09-11,Inyo,California,1599,39.00,6027,CA,County,18039,8864.13,216.20,216.20,8864.13,5,35,4.00,0.00
72605,2021-09-11,San Joaquin,California,94941,1573.00,6077,CA,County,762148,12457.03,206.39,206.39,12457.03,6,16,334.00,3.00
42192,2021-09-11,Riverside,California,352267,4826.00,6065,CA,County,2470546,14258.67,195.34,195.34,14258.67,7,7,1214.00,0.00
655773,2021-09-11,Merced,California,38304,527.00,6047,CA,County,277680,13794.30,189.79,189.79,13794.30,8,10,0.00,0.00
113334,2021-09-11,Tulare,California,70089,883.00,6107,CA,County,466195,15034.27,189.41,189.41,15034.27,9,5,366.00,3.00
41084,2021-09-11,Fresno,California,132281,1846.00,6019,CA,County,999101,13240.00,184.77,184.77,13240.00,10,13,619.00,8.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5422,09/07/21,Lassen,6444,29.00,21077.42,94.85,32,1,0.00,0.00
5423,09/08/21,Lassen,6501,29.00,21263.86,94.85,32,1,57.00,0.00
5424,09/09/21,Lassen,6501,29.00,21263.86,94.85,32,1,0.00,0.00
5425,09/10/21,Lassen,6577,31.00,21512.45,101.40,32,1,76.00,2.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5508,09/07/21,Fresno,130227,1828.00,13034.42,182.96,10,13,354.00,0.00
5509,09/08/21,Fresno,130757,1828.00,13087.47,182.96,10,13,530.00,0.00
5510,09/09/21,Fresno,131266,1831.00,13138.41,183.26,10,13,509.00,3.00
5511,09/10/21,Fresno,131662,1838.00,13178.05,183.97,10,13,396.00,7.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)